In [ ]:
# April 2025
# running shell command with agents

In [ ]:
from dotenv import load_dotenv
import os

# 1st method: using .env file.
load_dotenv()
# Access them using os.getenv or os.environ
api_key = os.getenv("GROQ_API_KEY")

# 2nd method: using hard code
# api_key = "<put the api key here>"
# if not os.environ.get("GROQ_API_KEY"):
#     os.environ["GROQ_API_KEY"] = api_key #getpass.getpass("Enter API key for Groq: ")




from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

In [ ]:
from langchain.agents import initialize_agent, AgentType
# from langchain.chat_models import ChatOpenAI
from langchain_community.tools.shell.tool import ShellTool

# Optional: to capture output instead of printing directly
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# # Initialize LLM (you can use ChatOpenAI or any other)
# llm = ChatOpenAI(temperature=0)

# Initialize the Shell tool
shell_tool = ShellTool()

# Create an agent that uses the shell tool
agent = initialize_agent(
    tools=[shell_tool],
    llm=llm,
    agent= AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    # verbose=True,
)

# Example command
agent.run("List all files in the current directory")


In [ ]:
# !pip install langchain-experimental

## output with parsing

In [ ]:
from langchain.agents import initialize_agent, AgentType, Tool
from langchain_community.tools.shell.tool import ShellTool
# from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
import json

# 🔹 1. Define a structured response schema
class DirectoryListing(BaseModel):
    files: list[str] = Field(description="List of file and directory names")

# 🔹 2. Create a parser for the schema
parser = PydanticOutputParser(pydantic_object=DirectoryListing)

# 🔹 3. Custom shell logic for structured output
def list_files(_):
    import os
    return {"files": os.listdir(".")}

# 🔹 4. Wrap the function as a LangChain Tool
list_files_tool = Tool(
    name="ListFilesTool",
    func=lambda _: json.dumps(list_files(None)),  # Convert the output to a JSON string
    description="Lists files in the current directory and returns them in structured JSON format."
)

# 🔹 5. Initialize ShellTool (for general shell access)
shell_tool = ShellTool()

# # 🔹 6. Initialize LLM
# llm = ChatOpenAI(temperature=0)

# 🔹 7. Build the agent with tools
agent = initialize_agent(
    tools=[list_files_tool, shell_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    # verbose=True
)

# 🔹 8. Run the agent with a natural prompt
response = agent.run("List all files in the current directory as JSON")
print(response)


In [ ]:
# !pip install langchain langchain-community
